In [ ]:
!pip install -U qiskit
!pip install pylatexenc
!pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 37.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 3.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=ceeb7220f5c8a2eb5e17ddf3a6960e273975e1f3b8bbca63dcfb313340de98dc
  Stored in directory: /root/.cache/pip/wheels/b1/7a/33/9fdd892f784ed4afda62b685ae3703adf4c91aa0f524c28f03
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 57.2 MB/s eta 0:00:00


In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer.primitives import Sampler
import numpy as np
import random

# Define registers
qr = QuantumRegister(2, name="qr")
cr = ClassicalRegister(4, name="cr")

# Create singlet (Bell state) preparation circuit
singlet = QuantumCircuit(qr, cr, name='singlet')
singlet.h(qr[0])
singlet.cx(qr[0], qr[1])

## Alice's measurement circuits

# measure the spin projection of Alice's qubit onto the a_1 direction (X basis)
measureA1 = QuantumCircuit(qr, cr, name='measureA1')
measureA1.h(qr[0])
measureA1.measure(qr[0],cr[0])

# measure the spin projection of Alice's qubit onto the a_2 direction (W basis)
measureA2 = QuantumCircuit(qr, cr, name='measureA2')
measureA2.s(qr[0])
measureA2.h(qr[0])
measureA2.t(qr[0])
measureA2.h(qr[0])
measureA2.measure(qr[0],cr[0])

# measure the spin projection of Alice's qubit onto the a_3 direction (standard Z basis)
measureA3 = QuantumCircuit(qr, cr, name='measureA3')
measureA3.measure(qr[0],cr[0])

## Bob's measurement circuits

# measure the spin projection of Bob's qubit onto the b_1 direction (W basis)
measureB1 = QuantumCircuit(qr, cr, name='measureB1')
measureB1.s(qr[1])
measureB1.h(qr[1])
measureB1.t(qr[1])
measureB1.h(qr[1])
measureB1.measure(qr[1],cr[1])

# measure the spin projection of Bob's qubit onto the b_2 direction (standard Z basis)
measureB2 = QuantumCircuit(qr, cr, name='measureB2')
measureB2.measure(qr[1],cr[1])

# measure the spin projection of Bob's qubit onto the b_3 direction (V basis)
measureB3 = QuantumCircuit(qr, cr, name='measureB3')
measureB3.s(qr[1])
measureB3.h(qr[1])
measureB3.tdg(qr[1])
measureB3.h(qr[1])
measureB3.measure(qr[1],cr[1])

## Lists of measurement circuits
aliceMeasurements = [measureA1, measureA2, measureA3]
bobMeasurements = [measureB1, measureB2, measureB3]

# Settings
number_of_key_bits = 512 * 512 * 3 * 8  # Desired number of bits
estimated_factor = 3
numberOfSinglets = 5000  # Batched size for testing, increase if needed
batch_size = 1000

# Generate random measurement choices
aliceChoices = [random.randint(1, 3) for _ in range(numberOfSinglets)]
bobChoices = [random.randint(1, 3) for _ in range(numberOfSinglets)]

# Initialize sampler and results
sampler = Sampler()
aliceResults, bobResults = [], []

# Run circuits in batches
for batch_start in range(0, numberOfSinglets, batch_size):
    batch_end = min(batch_start + batch_size, numberOfSinglets)
    circuits = []

    for i in range(batch_start, batch_end):
        base = singlet.copy()
        base = base.compose(aliceMeasurements[aliceChoices[i] - 1])
        base = base.compose(bobMeasurements[bobChoices[i] - 1])
        circuits.append(base)

    # Simulate the batch
    result = sampler.run(circuits).result()
    for dist in result.quasi_dists:
        measured = max(dist.items(), key=lambda x: x[1])[0]
        res = f"{int(measured):04b}"[-2:]  # Extract Alice and Bob's measured bits
        if res == '00':
            aliceResults.append(0)
            bobResults.append(0)
        elif res == '01':
            aliceResults.append(1)
            bobResults.append(0)
        elif res == '10':
            aliceResults.append(0)
            bobResults.append(1)
        elif res == '11':
            aliceResults.append(1)
            bobResults.append(1)

# Filter results where Alice and Bob used compatible bases
aliceKey = []
bobKey = []
for i in range(numberOfSinglets):
    if (aliceChoices[i] == 2 and bobChoices[i] == 1) or (aliceChoices[i] == 3 and bobChoices[i] == 2):
        aliceKey.append(aliceResults[i])
        bobKey.append(bobResults[i])

# Final matching key bits
finalKeyBits = [a for a, b in zip(aliceKey, bobKey) if a == b]

print("Total key bits extracted:", len(finalKeyBits))

# Pad or truncate to desired length
if len(finalKeyBits) < number_of_key_bits:
    print("Not enough key bits. Padding...")
    finalKeyBits = (finalKeyBits * ((number_of_key_bits // len(finalKeyBits)) + 1))[:number_of_key_bits]
else:
    finalKeyBits = finalKeyBits[:number_of_key_bits]

# Convert bits to hexadecimal byte format
hex_key_bytes = []
for i in range(0, len(finalKeyBits), 8):
    byte_bits = finalKeyBits[i:i+8]
    byte_value = 0
    for bit in byte_bits:
        byte_value = (byte_value << 1) | bit
    hex_key_bytes.append("{:02x}".format(byte_value))

# Save to file
with open("quantum_key_1.txt", "w") as f:
    for hex_byte in hex_key_bytes:
        f.write(hex_byte + "\n")

print("Quantum key written to quantum_key.txt with", len(hex_key_bytes), "bytes.")



Total key bits extracted: 1100
Not enough key bits. Padding...
Quantum key written to quantum_key.txt with 786432 bytes.
